<a href="https://colab.research.google.com/github/peco-glhf/ai_writer/blob/main/audio_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 保存先フォルダを作成（存在しない場合）
import os
drive_folder = '/content/drive/MyDrive/音声処理結果'
os.makedirs(drive_folder, exist_ok=True)

# ffmpegのインストール（すでにインストール済みでも念のため）
!apt install -y ffmpeg

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
# ファイルをアップロード
from google.colab import files
uploaded = files.upload()

# アップロードされたファイル名を取得（最初のファイルを使用）
input_filename = list(uploaded.keys())[0]


Saving naoya_audio.wav to naoya_audio.wav


In [ ]:
# 一意のフォルダ名を作成（オリジナルファイル名 + 日時）
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_basename = os.path.splitext(input_filename)[0]  # 拡張子なしのファイル名を取得
drive_folder = f'{base_folder}/{file_basename}_{timestamp}'
os.makedirs(drive_folder, exist_ok=True)

print(f"処理結果は以下のフォルダに保存されます: {drive_folder}")

# 必要なライブラリをインストール
!pip install noisereduce librosa soundfile pydub

# 音声処理ライブラリをインポート
import noisereduce as nr
import librosa
import soundfile as sf
from pydub import AudioSegment, effects

# アップロードされたファイルを一時的に保存
drive_audio_path = f'{drive_folder}/{input_filename}'
with open(input_filename, 'rb') as f_in:
    with open(drive_audio_path, 'wb') as f_out:
        f_out.write(f_in.read())

# 音声をロード（モノラル、16kHzで統一）
audio_data, rate = librosa.load(drive_audio_path, sr=16000)

# ノイズ除去（冒頭2秒間をノイズプロファイルとして使用）
noise_sample = audio_data[0:rate*2]
reduced_noise_audio = nr.reduce_noise(
    y=audio_data,
    y_noise=noise_sample,
    sr=rate,
    stationary=True
)

# ノイズ除去した音声をGoogle Driveに保存
drive_clean_path = f'{drive_folder}/clean_{file_basename}.wav'
sf.write(drive_clean_path, reduced_noise_audio, rate)

# 音量正規化（pydub）
audio_segment = AudioSegment.from_wav(drive_clean_path)
normalized_audio = effects.normalize(audio_segment)

# 最終処理済み音声をGoogle Driveに保存
drive_final_path = f'{drive_folder}/final_{file_basename}.wav'
normalized_audio.export(drive_final_path, format='wav')

print(f"処理完了！")
print(f"オリジナルファイル: {drive_audio_path}")
print(f"ノイズ除去済みファイル: {drive_clean_path}")
print(f"最終処理済みファイル: {drive_final_path}")

処理完了！最終ファイルの保存先: /content/drive/MyDrive/音声処理結果/final_audio.wav


In [ ]:
# @title
# 音声をColab上で確認
from IPython.display import Audio
print("処理後の音声を再生：")
display(Audio(drive_final_path))

# 保存場所の情報を表示
print("\n全ての処理が完了しました。")
print(f"保存先フォルダ：{drive_folder}")


print(f"元の音声ファイル: {drive_audio_path}")
print(f"ノイズ除去後の音声: {drive_clean_path}")
print(f"最終処理済み音声: {drive_final_path}")